In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
1,Trump's doctors say he could go home Monday. O...,https://www.usatoday.com/story/news/health/202...,"Sun, 04 Oct 2020 19:41:15 GMT",2020-10-04 19:41:15+00:00
7,Newsom Signs COVID-19 Protections For Agricult...,http://www.capradio.org/articles/2020/10/04/ne...,"Sun, 04 Oct 2020 19:03:23 GMT",2020-10-04 19:03:23+00:00
2,Only 3 states are reporting declines in new co...,https://www.cnn.com/2020/10/04/health/us-coron...,"Sun, 04 Oct 2020 18:43:00 GMT",2020-10-04 18:43:00+00:00
9,"Cam Newton, New England Patriots quarterback, ...",https://www.cnn.com/2020/10/03/us/cam-newton-c...,"Sun, 04 Oct 2020 17:49:00 GMT",2020-10-04 17:49:00+00:00
0,Governor Cuomo Updates New Yorkers on State's ...,https://www.governor.ny.gov/news/governor-cuom...,"Sun, 04 Oct 2020 16:51:12 GMT",2020-10-04 16:51:12+00:00
8,Countries across Europe set grim Covid-19 reco...,https://www.cnn.com/2020/10/04/europe/europe-c...,"Sun, 04 Oct 2020 12:09:00 GMT",2020-10-04 12:09:00+00:00
5,"Is It COVID-19, Allergies, The Flu Or A Cold? ...",https://www.npr.org/sections/health-shots/2020...,"Sun, 04 Oct 2020 11:00:00 GMT",2020-10-04 11:00:00+00:00
6,Tracking COVID-19 in Nebraska KOLN,https://www.1011now.com/2020/10/03/tracking-co...,"Sat, 03 Oct 2020 21:16:00 GMT",2020-10-03 21:16:00+00:00
4,Here's what is known about Trump's COVID-19 tr...,https://www.sciencemag.org/news/2020/10/heres-...,"Sat, 03 Oct 2020 01:25:00 GMT",2020-10-03 01:25:00+00:00
3,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Wed, 30 Sep 2020 21:00:00 GMT",2020-09-30 21:00:00+00:00
